<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/NLP_Supervised_Project/2_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. VECTORIZATION

## 2.1 Imports

In [1]:
from google.colab import files
from google.colab import drive
import pandas as pd
import nltk
import io
import re
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
drive.mount('/drive')

Mounted at /drive


## 2.2 Importing dataset

The dataset that it is going to take for this phase is called **processed_tweets.csv**

In [3]:
def upload_dataframes (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

In [4]:
tweets = upload_dataframes([])

Saving processed_tweets.csv to processed_tweets.csv
User uploaded file "processed_tweets.csv" with length 1558482 bytes


In [5]:
print(tweets.shape)
tweets.head()

(24783, 7)


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt woman complain cleaning house man always ta...
1,1,3,0,3,0,1,rt boy dat coldtyga dwn bad cuffin dat hoe 1st...
2,2,3,0,3,0,1,rt dawg rt ever fuck bitch start cry confused ...
3,3,3,0,2,1,1,rt look like tranny
4,4,6,0,6,0,1,rt shit hear might true might faker bitch told...


## 2.3 TFIDF

In [6]:
def tfidf_vectorizer(matrix):
  vectorizer = TfidfVectorizer(min_df=1, lowercase=False)
  return vectorizer.fit_transform(matrix)

In [29]:
tweets_tfidf = tfidf_vectorizer(tweets['tweet'])
tweets_tfidf

<24783x21378 sparse matrix of type '<class 'numpy.float64'>'
	with 185156 stored elements in Compressed Sparse Row format>

## 2.3 TFIDF with N-grams

In [9]:
def tfidf_ngram_vectorizer(matrix):
  vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1, lowercase=False)
  return vectorizer.fit_transform(matrix)

In [10]:
tweets_ngrams = tfidf_ngram_vectorizer(tweets['tweet'])
tweets_ngrams

<24783x135324 sparse matrix of type '<class 'numpy.float64'>'
	with 351455 stored elements in Compressed Sparse Row format>

## 2.3 TFIDF with N-grams and POS-tagging

In [11]:
def tokenize_pos(raw):
  tokens = word_tokenize(raw)
  tags = nltk.pos_tag(tokens)
  raw_tags = " ".join(tag for (word, tag) in tags)
  return raw_tags

In [13]:
tokens_tweet = tweets['tweet'].apply(tokenize_pos)
tokens_tweet.head()

0          NNS NN VBP NN NN NN RB VBP NN
1       NN NN NN NN NN JJ NN NN NN CD NN
2        NN NN NN RB VBD JJ NN NN VBD NN
3                            JJ NN IN NN
4    NN VBD JJ MD JJ MD VB NN VBD CD NNS
Name: tweet, dtype: object

In [14]:
tweets_ngrams_pos = tfidf_ngram_vectorizer(tokens_tweet)
tweets_ngrams_pos

<24783x541 sparse matrix of type '<class 'numpy.float64'>'
	with 221645 stored elements in Compressed Sparse Row format>

## 2.4 TFIDF with N-grams, POS-tagging and other features.

### Number of RT's

In [15]:
def num_rt(raw):
  rt = 0
  words = raw.split()
  for word in words:
    if word == 'rt':
      rt += 1
  return rt

In [16]:
tweets_rt = tweets['tweet'].apply(num_rt)
tweets_rt.head()

0    1
1    1
2    2
3    1
4    1
Name: tweet, dtype: int64

In [17]:
tweets_rt = np.reshape(tweets_rt.to_list(), (len(tweets_rt), 1))
tweets_rt

array([[1],
       [1],
       [2],
       ...,
       [0],
       [0],
       [0]])

In [18]:
tweets_rt = sp.csr_matrix(tweets_rt)
tweets_rt

<24783x1 sparse matrix of type '<class 'numpy.longlong'>'
	with 7130 stored elements in Compressed Sparse Row format>

###Sentiment Analysis

In [19]:
def sentiment_analysis(raw):
  sentiment_analyzer  = SentimentIntensityAnalyzer() 
  sentiment = sentiment_analyzer.polarity_scores(raw)
  return [value for key, value in sentiment.items()]

In [20]:
tweets_sentiment = tweets['tweet'].apply(sentiment_analysis)
tweets_sentiment.head()

0     [0.238, 0.762, 0.0, -0.3612]
1     [0.259, 0.741, 0.0, -0.5423]
2      [0.765, 0.235, 0.0, -0.946]
3      [0.0, 0.545, 0.455, 0.3612]
4    [0.43, 0.407, 0.163, -0.6808]
Name: tweet, dtype: object

In [23]:
tweets_sentiment = sp.csr_matrix(tweets_sentiment.to_list())
tweets_sentiment

<24783x4 sparse matrix of type '<class 'numpy.float64'>'
	with 72757 stored elements in Compressed Sparse Row format>

###Hatred N-gram dictionary

In [24]:
hatred_dictionary = upload_dataframes([])

Saving refined_ngram_dict.csv to refined_ngram_dict.csv
User uploaded file "refined_ngram_dict.csv" with length 3178 bytes


In [25]:
hatred_dictionary.head()

,ngram,prophate
0,allah akbar,0.870
1,blacks,0.583
2,chink,0.467
3,chinks,0.542
4,dykes,0.602


In [26]:
def hatred_dict_analysis(raw):
  weights = []
  for i in hatred_dictionary.index:
    if re.search(hatred_dictionary['ngram'][i], raw) is None:
      weights.append(0.0)
    else:
      weights.append(hatred_dictionary['prophate'][i])
  return weights

In [27]:
tweets_hatred_dict = tweets['tweet'].apply(hatred_dict_analysis)
tweets_hatred_dict.head()

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: tweet, dtype: object

In [28]:
tweets_hatred_dict = sp.csr_matrix(tweets_hatred_dict.to_list())
tweets_hatred_dict

<24783x178 sparse matrix of type '<class 'numpy.float64'>'
	with 1367 stored elements in Compressed Sparse Row format>

##2.5 Unify all configurations

In [31]:
configurations = sp.hstack((tweets_tfidf, tweets_ngrams, tweets_ngrams_pos,
                            tweets_rt, tweets_sentiment, tweets_hatred_dict), format='csr')
configurations

<24783x157426 sparse matrix of type '<class 'numpy.float64'>'
	with 839510 stored elements in Compressed Sparse Row format>